# **Recap of [Lesson 7](https://colab.research.google.com/drive/1LokUcM5YSazdq5jfbkop-Z5rmP-39y4r#forceEdit=true&sandboxMode=true) 👩‍💻🚀**

1. Learned what is a schema.
2. Explored schema settings and components.
3. Learned how to retrieve dlt pipeline schema.
4. Learned how to adjust schema.

---

# **Understanding Pipeline Metadata and State** 👻📄 [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_8_understanding_pipeline_metadata_and_state.ipynb) [![GitHub badge](https://img.shields.io/badge/github-view_source-2b3137?logo=github)](https://github.com/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_8_understanding_pipeline_metadata_and_state.ipynb)


**Here, you will learn or brush up on:**
- What's pipeline metadata
- Exploring pipeline metadata from load info
- Exploring pipeline metadate from trace
- Exploring pipeline metadata from state

---
##  **Pipeline Metadata**


Metadata is basically data about data.

Pipeline Metadata is data about your data pipeline. This can be useful if you want to know things like:

- When your pipeline first ran
- When your pipeline last ran
- Information about your source or destination
- Processing time
- Or information that you yourself may want to add to the metadata
- And much more!


![Lesson_8_Understanding_Pipeline_Metadata_and_State_img1](https://storage.googleapis.com/dlt-blog-images/dlt-fundamentals-course/Lesson_8_Understanding_Pipeline_Metadata_and_State_img1.jpeg)

 `dlt` allows you to be able to view all this metadata through various options!

This notebook will walk you through those options. Namely:

- Load info
- Trace
- State

Let's load some GitHub data to DuckDB to inspect the pipeline metadata in different ways. First we need to install dlt with DuckDB:

In [2]:
%%capture
!pip install "dlt[duckdb]"

Define a dlt resource that fetches Pull Requests and wrap it in a dlt source:

In [3]:
from typing import Iterable
import dlt
from dlt.extract import DltResource
from dlt.common.typing import TDataItems
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator

import os
from google.colab import userdata

os.environ["SOURCES__SECRET_KEY"] = userdata.get("SECRET_KEY")



@dlt.source
def github_source(secret_key: str = dlt.secrets.value) -> Iterable[DltResource]:
    client = RESTClient(
        base_url="https://api.github.com",
        auth=BearerTokenAuth(token=secret_key),
        paginator=HeaderLinkPaginator(),
    )

    @dlt.resource
    def github_pulls(
        cursor_date: dlt.sources.incremental[str] = dlt.sources.incremental(
            "updated_at", initial_value="2024-12-01"
        )
    ) -> TDataItems:
        params = {"since": cursor_date.last_value, "status": "open"}
        for page in client.paginate("repos/dlt-hub/dlt/pulls", params=params):
            yield page

    return github_pulls


# define new dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="github_pipeline",
    destination="duckdb",
    dataset_name="github_data",
)


# run the pipeline with the new resource
load_info = pipeline.run(github_source())
print(load_info)

2025-12-31 12:49:32,714|[WARNING]|260|134477189058560|dlt|validate.py|verify_normalized_table:91|In schema `github_source`: The following columns in table 'github_pulls' did not receive any data during this load and therefore could not have their types inferred:
  - closed_at
  - merged_at
  - assignee
  - milestone
  - head__repo__mirror_url
  - base__repo__mirror_url
  - auto_merge
  - active_lock_reason

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'closed_at': {'data_type': 'text'}})



Pipeline github_pipeline load step completed in 0.85 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data
Load package 1767185371.47367 is LOADED and contains no failed jobs


---
##  **Load info**


`Load Info:` This is a collection of useful information about the recently loaded data. It includes details like the pipeline and dataset name, destination information, and a list of loaded packages with their statuses, file sizes, types, and error messages (if any).

`Load Package:` A load package is a collection of jobs with data for specific tables, generated during each execution of the pipeline. Each package is uniquely identified by a `load_id`.

---
###  **(0) CLI**


From the [`Inspecting & Adjusting Schema`](https://colab.research.google.com/drive/1LokUcM5YSazdq5jfbkop-Z5rmP-39y4r) Colab we've already learned that we can see which schema changes a load package has introduced with the command:

```
dlt pipeline -v <pipeline_name> load-package
```

The verbose flag only accounts for the schema changes, so if we run it without the flag, we will still see the most recent load package info:

In [6]:
!dlt pipeline github_pipeline load-package

Found pipeline github_pipeline in /var/dlt/pipelines
Package 1767168187.9246864 found in /var/dlt/pipelines/github_pipeline/load/loaded/1767168187.9246864
The package with load id 1767168187.9246864 for schema github_source is in LOADED state. It updated schema for 9 tables. The package was LOADED at 2025-12-31 08:03:10.116288+00:00.
Jobs details:
Job: _dlt_pipeline_state.31e163b1b6.insert_values.gz, table: _dlt_pipeline_state in completed_jobs. File type: insert_values, size: 618B. Started on: 2025-12-31 08:03:09.184203+00:00 and completed in 0.93 seconds.
Job: github_pulls__assignees.8a78cff0b6.insert_values.gz, table: github_pulls__assignees in completed_jobs. File type: insert_values, size: 2.1K. Started on: 2025-12-31 08:03:09.235208+00:00 and completed in 0.88 seconds.
Job: github_pulls__base__repo__topics.5fbb419986.insert_values.gz, table: github_pulls__base__repo__topics in completed_jobs. File type: insert_values, size: 7.8K. Started on: 2025-12-31 08:03:09.233208+00:00 and c

The `load_id` of a particular package is added to the top data tables (parent tables) and to the special `_dlt_loads` table with a status of 0 when the load process is fully completed. The `_dlt_loads` table tracks complete loads and allows chaining transformations on top of them.

We can also see load package info with a specific load id:

In [8]:
!dlt pipeline github_pipeline load-package 1767168187.9246864

Found pipeline github_pipeline in /var/dlt/pipelines
Package 1767168187.9246864 found in /var/dlt/pipelines/github_pipeline/load/loaded/1767168187.9246864
The package with load id 1767168187.9246864 for schema github_source is in LOADED state. It updated schema for 9 tables. The package was LOADED at 2025-12-31 08:03:10.116288+00:00.
Jobs details:
Job: _dlt_pipeline_state.31e163b1b6.insert_values.gz, table: _dlt_pipeline_state in completed_jobs. File type: insert_values, size: 618B. Started on: 2025-12-31 08:03:09.184203+00:00 and completed in 0.93 seconds.
Job: github_pulls__assignees.8a78cff0b6.insert_values.gz, table: github_pulls__assignees in completed_jobs. File type: insert_values, size: 2.1K. Started on: 2025-12-31 08:03:09.235208+00:00 and completed in 0.88 seconds.
Job: github_pulls__base__repo__topics.5fbb419986.insert_values.gz, table: github_pulls__base__repo__topics in completed_jobs. File type: insert_values, size: 7.8K. Started on: 2025-12-31 08:03:09.233208+00:00 and c

---
###  **(0) Python**


From the [`Inspecting & Adjusting Schema`](https://colab.research.google.com/drive/1LokUcM5YSazdq5jfbkop-Z5rmP-39y4r?usp=sharing) Colab we've also learned that a schema can be accessed with:

```python
print(load_info.load_packages[0].schema)
```
Similarly if we drop the schema part, we will just get the load package info:

In [4]:
print(load_info.load_packages[0])

The package with load id 1767185371.47367 for schema github_source is in LOADED state. It updated schema for 9 tables. The package was LOADED at 2025-12-31 12:49:33.607081+00:00.
Jobs details:
Job: github_pulls__head__repo__topics.0b568cbe62.insert_values.gz, table: github_pulls__head__repo__topics in completed_jobs. File type: insert_values, size: 5.4K. Started on: 2025-12-31 12:49:32.701979+00:00 and completed in 0.91 seconds.
Job: github_pulls__assignees.bbd2c3c64a.insert_values.gz, table: github_pulls__assignees in completed_jobs. File type: insert_values, size: 2.1K. Started on: 2025-12-31 12:49:32.708980+00:00 and completed in 0.90 seconds.
Job: github_pulls.b8fd73077b.insert_values.gz, table: github_pulls in completed_jobs. File type: insert_values, size: 44.6K. Started on: 2025-12-31 12:49:32.698979+00:00 and completed in 0.91 seconds.
Job: github_pulls__labels.32c1711303.insert_values.gz, table: github_pulls__labels in completed_jobs. File type: insert_values, size: 1002B. Sta

which has the following public methods and attributes:

In [5]:
# This code snippet just prints out the public methoda and attributes of the schema object in load info
all_attributes_methods = dir(load_info.load_packages[0])
public_attributes_methods = [
    attr for attr in all_attributes_methods if not attr.startswith("_")
]

print(f"{'Attribute/Method':<50} {'Type':<10}")
print("-" * 40)
for attr in public_attributes_methods:
    attr_value = getattr(load_info.load_packages[0], attr)
    if callable(attr_value):
        print(f"{attr:<50} {'method':<10}")
    else:
        print(f"{attr:<50} {'attribute':<10}")

Attribute/Method                                   Type      
----------------------------------------
asdict                                             method    
asstr                                              method    
completed_at                                       attribute 
count                                              method    
index                                              method    
jobs                                               attribute 
load_id                                            attribute 
package_path                                       attribute 
schema                                             attribute 
schema_hash                                        attribute 
schema_name                                        attribute 
schema_update                                      attribute 
state                                              attribute 


---
##  **Trace**


 `Trace`: A trace is a detailed record of the execution of a pipeline. It provides rich information on the pipeline processing steps: **extract**, **normalize**, and **load**. It also shows the last `load_info`.

---
###  **(0) CLI**


You can access pipeline trace using the command:


```
dlt pipeline <pipeline_name> trace
```

Try on the github issues pipeline:

In [6]:
!dlt pipeline github_pipeline trace

Found pipeline github_pipeline in /var/dlt/pipelines
Run started at 2025-12-31 12:49:30.643084+00:00 and COMPLETED in 3.82 seconds with 4 steps.
Step extract COMPLETED in 0.97 seconds.

Load package 1767185371.47367 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.29 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- github_pulls: 45 row(s)
- github_pulls__head__repo__topics: 310 row(s)
- github_pulls__base__repo__topics: 450 row(s)
- github_pulls__assignees: 30 row(s)
- github_pulls__requested_reviewers: 20 row(s)
- github_pulls__labels: 12 row(s)

Load package 1767185371.47367 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 1.73 seconds.
Pipeline github_pipeline load step completed in 0.85 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipel

---
###  **(0) Python**


We can also print out the trace in code:

In [15]:
# print human friendly trace information
print(pipeline.last_trace)

Run started at 2025-12-31 08:03:07.023104+00:00 and COMPLETED in 3.92 seconds with 4 steps.
Step extract COMPLETED in 1.17 seconds.

Load package 1767168187.9246864 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.17 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- github_pulls: 45 row(s)
- github_pulls__head__repo__topics: 310 row(s)
- github_pulls__base__repo__topics: 450 row(s)
- github_pulls__assignees: 30 row(s)
- github_pulls__requested_reviewers: 20 row(s)
- github_pulls__labels: 12 row(s)

Load package 1767168187.9246864 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 1.68 seconds.
Pipeline github_pipeline load step completed in 0.84 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data
Load package 17

Separately receive the extract stage info:

In [17]:
# print human friendly trace information
print(pipeline.last_trace.last_extract_info)


Load package 1767168187.9246864 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs


As well as the normalization stage info with:

In [18]:
# print human friendly normalization information
print(pipeline.last_trace.last_normalize_info)

Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- github_pulls: 45 row(s)
- github_pulls__head__repo__topics: 310 row(s)
- github_pulls__base__repo__topics: 450 row(s)
- github_pulls__assignees: 30 row(s)
- github_pulls__requested_reviewers: 20 row(s)
- github_pulls__labels: 12 row(s)

Load package 1767168187.9246864 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs


In particular how many rows of data were normalized:

In [19]:
# access row counts dictionary of normalize info
print(pipeline.last_trace.last_normalize_info.row_counts)

{'_dlt_pipeline_state': 1, 'github_pulls': 45, 'github_pulls__head__repo__topics': 310, 'github_pulls__base__repo__topics': 450, 'github_pulls__assignees': 30, 'github_pulls__requested_reviewers': 20, 'github_pulls__labels': 12}


And finally the load stage info:

In [20]:
# print human friendly load information
print(pipeline.last_trace.last_load_info)

Pipeline github_pipeline load step completed in 0.84 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data
Load package 1767168187.9246864 is LOADED and contains no failed jobs


---
##  **State**

[`The pipeline state`](https://dlthub.com/docs/general-usage/state) is a Python dictionary that lives alongside your data. You can store values in it during a pipeline run, and then retrieve them in the next pipeline run. It's used for tasks like preserving the "last value" or similar loading checkpoints, and it gets committed atomically with the data. The state is stored locally in the pipeline working directory and is also stored at the destination for future runs.


**When to use pipeline state**
- dlt uses the state internally to implement last value incremental loading. This use case should cover around 90% of your needs to use the pipeline state.
- Store a list of already requested entities if the list is not much bigger than 100k elements.
- Store large dictionaries of last values if you are not able to implement it with the standard incremental construct.
- Store the custom fields dictionaries, dynamic configurations and other source-scoped state.

**When not to use pipeline state**

Do not use dlt state when it may grow to millions of elements. Do you plan to store modification timestamps of all of your millions of user records? This is probably a bad idea! In that case you could:

- Store the state in dynamo-db, redis etc. taking into the account that if the extract stage fails you'll end with invalid state.
- Use your loaded data as the state. dlt exposes the current pipeline via dlt.current.pipeline() from which you can obtain sqlclient and load the data of interest. In that case try at least to process your user records in batches.

---
###  **(0) CLI**


In [7]:
!dlt pipeline -v github_pipeline info

Attaching to pipeline github_pipeline
Found pipeline github_pipeline in /var/dlt/pipelines
Synchronized state:
_state_version: 1
_state_engine_version: 4
dataset_name: github_data
schema_names: ['github_source']
pipeline_name: github_pipeline
default_schema_name: github_source
destination_type: dlt.destinations.duckdb
destination_name: None
_version_hash: nrWYbJwNvAkt17uM8D4hgn4c+uMiz6N5pMAu5BAaNWs=

sources:
{
  "github_source": {
    "resources": {
      "github_pulls": {
        "incremental": {
          "updated_at": {
            "initial_value": "2024-12-01",
            "last_value": "2025-12-30T13:39:21Z",
            "unique_hashes": [
              "pkjQiUBfoMIwJumCQqtv"
            ]
          }
        }
      }
    }
  }
}

Local state:
first_run: False
initial_cwd: /content
last_run_context['uri']: file:///content
_last_extracted_at: 2025-12-31 12:49:32.432102+00:00
_last_extracted_hash: nrWYbJwNvAkt17uM8D4hgn4c+uMiz6N5pMAu5BAaNWs=

Resources in schema: github_source
git

---
###  **(1) Python**


In [13]:
import json


def read_state(filepath: str) -> str:
    with open(filepath, "r", encoding="utf-8") as file:
        data = json.load(file)
        pretty_json = json.dumps(data, indent=4)
        return pretty_json

In [26]:
# stored in your default pipelines folder
print(read_state("/var/dlt/pipelines/github_pipeline/state.json"))

{
    "_state_version": 2,
    "_state_engine_version": 4,
    "_local": {
        "first_run": false,
        "initial_cwd": "/content",
        "last_run_context": {
            "settings_dir": "/content/.dlt",
            "local_dir": "/content",
            "run_dir": "/content",
            "uri": "file:///content"
        },
        "_last_extracted_at": "\uf0272025-12-31T10:07:31.139913+00:00",
        "_last_extracted_hash": "4oOzor7aQLoJOiydgMkQbNK1ilzeqHySrOWTQ99bMw0="
    },
    "schema_names": [
        "github_source"
    ],
    "dataset_name": "github_data",
    "pipeline_name": "github_pipeline",
    "default_schema_name": "github_source",
    "destination_type": "dlt.destinations.duckdb",
    "destination_name": null,
    "_version_hash": "4oOzor7aQLoJOiydgMkQbNK1ilzeqHySrOWTQ99bMw0=",
    "sources": {
        "github_source": {
            "resources": {
                "github_pulls": {
                    "incremental": {
                        "updated_at": {
     

---
###  **Modify State**

The pipeline state is a Python dictionary that lives alongside your data; you can store values in it and, on the next pipeline run, request them back.

---
####  **(0) Resource state**

You can **read** and **write** the state in your resources using:

```python
dlt.current.resource_state().get()
```
and

```python
dlt.current.resource_state().setdefault(key, value)
```

In [9]:
import dlt
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator

import os
from google.colab import userdata

os.environ["SOURCES__SECRET_KEY"] = userdata.get("SECRET_KEY")


@dlt.source
def github_source(secret_key: str = dlt.secrets.value) -> Iterable[DltResource]:
    client = RESTClient(
        base_url="https://api.github.com",
        auth=BearerTokenAuth(token=secret_key),
        paginator=HeaderLinkPaginator(),
    )

    @dlt.resource
    def github_pulls(
        cursor_date: dlt.sources.incremental[str] = dlt.sources.incremental(
            "updated_at", initial_value="2024-12-01"
        )
    ) -> TDataItems:
        # Let's set some custom state information
        dlt.current.resource_state().setdefault(
            "new_key", ["first_value", "second_value"]
        )  # <--- new item in the state

        params = {"since": cursor_date.last_value, "status": "open"}
        for page in client.paginate("repos/dlt-hub/dlt/pulls", params=params):
            yield page

    return github_pulls


# define new dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="github_pipeline",
    destination="duckdb",
    dataset_name="github_data",
)


# run the pipeline with the new resource
load_info = pipeline.run(github_source())
print(load_info)

MessageError: FetchError: Could not fetch resource at https://colab.research.google.com/userdata/get?authuser=0&notebookid=https%3A%2F%2Fgithub.com%2Fdlt-hub%2Fdlt%2Fblob%2Fmaster%2Fdocs%2Feducation%2Fdlt-fundamentals-course%2Flesson_8_understanding_pipeline_metadata_and_state.ipynb&key=SECRET_KEY: 401  

In [10]:
print(read_state("/var/dlt/pipelines/github_pipeline/state.json"))

{
    "_state_version": 1,
    "_state_engine_version": 4,
    "_local": {
        "first_run": false,
        "initial_cwd": "/content",
        "last_run_context": {
            "settings_dir": "/content/.dlt",
            "local_dir": "/content",
            "run_dir": "/content",
            "uri": "file:///content"
        },
        "_last_extracted_at": "\uf0272025-12-31T12:49:32.432102+00:00",
        "_last_extracted_hash": "nrWYbJwNvAkt17uM8D4hgn4c+uMiz6N5pMAu5BAaNWs="
    },
    "dataset_name": "github_data",
    "schema_names": [
        "github_source"
    ],
    "pipeline_name": "github_pipeline",
    "default_schema_name": "github_source",
    "destination_type": "dlt.destinations.duckdb",
    "destination_name": null,
    "_version_hash": "nrWYbJwNvAkt17uM8D4hgn4c+uMiz6N5pMAu5BAaNWs=",
    "sources": {
        "github_source": {
            "resources": {
                "github_pulls": {
                    "incremental": {
                        "updated_at": {
     

In the state you will see the new items:

![Lesson_8_Understanding_Pipeline_Metadata_and_State_img2](https://storage.googleapis.com/dlt-blog-images/dlt-fundamentals-course/Lesson_8_Understanding_Pipeline_Metadata_and_State_img2.png)

You can modify any item in the state dict:

```python
new_keys = dlt.current.resource_state().setdefault("new_key", ["first_value", "second_value"])

if "something_happend":
    new_keys.append("third_value")

incremental_dict = dlt.current.resource_state().get("incremental")
incremental_dict.update({"second_new_key": "fourth_value"})
```

Full example:

In [14]:
import dlt
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator

import os
from google.colab import userdata

os.environ["SOURCES__SECRET_KEY"] = userdata.get("SECRET_KEY")


@dlt.source
def github_source(secret_key: str = dlt.secrets.value) -> Iterable[DltResource]:
    client = RESTClient(
        base_url="https://api.github.com",
        auth=BearerTokenAuth(token=secret_key),
        paginator=HeaderLinkPaginator(),
    )

    @dlt.resource
    def github_pulls(
        cursor_date: dlt.sources.incremental[str] = dlt.sources.incremental(
            "updated_at", initial_value="2024-12-01"
        )
    ) -> TDataItems:
        # Play with state even more
        new_keys = dlt.current.resource_state().setdefault(
            "new_key", ["first_value", "second_value"]
        )

        if "something_happened":
            new_keys.append("third_value")

        incremental_dict = dlt.current.resource_state().get("incremental")
        incremental_dict.update({"second_new_key": "fourth_value"})

        params = {"since": cursor_date.last_value, "status": "open"}
        for page in client.paginate("repos/dlt-hub/dlt/pulls", params=params):
            yield page

    return github_pulls


# define new dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="github_pipeline",
    destination="duckdb",
    dataset_name="github_data",
)


# run the pipeline with the new resource
load_info = pipeline.run(github_source())
print(load_info)

Pipeline github_pipeline load step completed in 0.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data
Load package 1767186321.0472355 is LOADED and contains no failed jobs


In [15]:
print(read_state("/var/dlt/pipelines/github_pipeline/state.json"))

{
    "_state_version": 3,
    "_state_engine_version": 4,
    "_local": {
        "first_run": false,
        "initial_cwd": "/content",
        "last_run_context": {
            "settings_dir": "/content/.dlt",
            "local_dir": "/content",
            "run_dir": "/content",
            "uri": "file:///content"
        },
        "_last_extracted_at": "\uf0272025-12-31T13:05:22.052194+00:00",
        "_last_extracted_hash": "Ayl1W2Pj2aOzKb3gc2O7UWjqxVRpxoAIQ+4xxQjZ1IE="
    },
    "dataset_name": "github_data",
    "schema_names": [
        "github_source"
    ],
    "pipeline_name": "github_pipeline",
    "default_schema_name": "github_source",
    "destination_type": "dlt.destinations.duckdb",
    "destination_name": null,
    "_version_hash": "Ayl1W2Pj2aOzKb3gc2O7UWjqxVRpxoAIQ+4xxQjZ1IE=",
    "sources": {
        "github_source": {
            "resources": {
                "github_pulls": {
                    "incremental": {
                        "updated_at": {
     

---
####  **(1) Source state**


You can also access the source-scoped state with `dlt.current.source_state()` which can be shared across resources of a particular source and is also available **read-only** in the source-decorated functions. The most common use case for the source-scoped state is to store mapping of custom fields to their displayable names.

Let's read some custom keys from the state:
```python
# Let's read some custom state information
source_new_keys = dlt.current.source_state().get("resources", {}).get("github_pulls", {}).get("new_key")
```
Full example:

In [16]:
import dlt
from dlt.sources.helpers import requests
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import BearerTokenAuth
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator

import os
from google.colab import userdata

os.environ["SOURCES__SECRET_KEY"] = userdata.get("SECRET_KEY")


@dlt.source
def github_source(secret_key: str = dlt.secrets.value) -> Iterable[DltResource]:
    client = RESTClient(
        base_url="https://api.github.com",
        auth=BearerTokenAuth(token=secret_key),
        paginator=HeaderLinkPaginator(),
    )

    @dlt.resource
    def github_pulls(
        cursor_date: dlt.sources.incremental[str] = dlt.sources.incremental(
            "updated_at", initial_value="2024-12-01"
        )
    ) -> TDataItems:
        params = {"since": cursor_date.last_value, "status": "open"}
        for page in client.paginate("repos/dlt-hub/dlt/pulls", params=params):
            yield page

        # Let's read some custom state information
        source_new_keys = (
            dlt.current.source_state()
            .get("resources", {})
            .get("github_pulls", {})
            .get("new_key")
        )
        print("My custom values: ", source_new_keys)

    return github_pulls


# define new dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="github_pipeline",
    destination="duckdb",
    dataset_name="github_data",
)


# run the pipeline with the new resource
load_info = pipeline.run(github_source())
print(load_info)

My custom values:  ['first_value', 'second_value', 'third_value', 'third_value']
Pipeline github_pipeline load step completed in ---
0 load package(s) were loaded to destination duckdb and into dataset None
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data


---
###  **Sync State**


What if you run your pipeline on, for example, Airflow where every task gets a clean filesystem and pipeline working directory is always deleted?

**dlt loads** your **state** into the destination **together** with all other **data** and when faced with a clean start, it will try to restore state from the destination.

The remote state is identified by pipeline name, the destination location (as given by the credentials) and destination dataset. To re-use **the same state**, use **the same pipeline name** and destination.

The state is stored in the `_dlt_pipeline_state` table at the destination and contains information about the pipeline, pipeline run (that the state belongs to) and state blob.

dlt has `dlt pipeline <pipeline name> sync` command where you can request the state back from that table.

💡 If you can keep the pipeline working directory across the runs, you can disable the state sync by setting `restore_from_destination=false` i.e. in your `config.toml`.

In [17]:
import duckdb
from google.colab import data_table
from IPython.display import display

data_table.enable_dataframe_formatter()

# a database 'chess_pipeline.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM _dlt_pipeline_state").df()
display(stats_table)

,version,engine_version,pipeline_name,state,created_at,version_hash,_dlt_load_id,_dlt_id
0,1,4,github_pipeline,eNplUV1PwjAU/S99dRA2hh9LfAB9kWQkJBqihjSX7cIqXS...,2025-12-31 12:49:32.407222+00:00,nrWYbJwNvAkt17uM8D4hgn4c+uMiz6N5pMAu5BAaNWs=,1767185371.47367,M/xMIgSQEB4D3Q
1,2,4,github_pipeline,eNplUdFOwjAU/Zc+D7INjGGJL/qiKJJFjImGNN12x+q6br...,2025-12-31 13:04:52.442745+00:00,4oOzor7aQLoJOiydgMkQbNK1ilzeqHySrOWTQ99bMw0=,1767186291.5332482,OaQdeulh7piNIQ
2,3,4,github_pipeline,eNptUdFOwjAU/Ze+OgjbMMYlPqDxAY3BGdEEQpqy3bGOrh...,2025-12-31 13:05:22.011923+00:00,Ayl1W2Pj2aOzKb3gc2O7UWjqxVRpxoAIQ+4xxQjZ1IE=,1767186321.0472355,666GzKM3GFymQg


Column "state" is compressed json dictionary.

|index|version|engine\_version|pipeline\_name|state|created\_at|version\_hash|\_dlt\_load\_id|\_dlt\_id|
|---|---|---|---|---|---|---|---|---|
|0|1|4|github\_pipeline|eNplkN....6+/m/QA7mbNc|2025-03-10 14:02:34\.340458+00:00|pnp+9AIA5jAGx5LKon6zWmPnfYVb10ROa5aIKjv9O0I=|1741615353\.5473728|FOzn5XuSZ/y/BQ|

In [27]:
!dlt --non-interactive pipeline github_pipeline info #--help
#dev_mode sync

NOTE: Non interactive mode. Default choices are automatically made for confirmations and prompts.
Found pipeline github_pipeline in /var/dlt/pipelines
Synchronized state:
_state_version: 3
_state_engine_version: 4
dataset_name: github_data
schema_names: ['github_source']
pipeline_name: github_pipeline
default_schema_name: github_source
destination_type: dlt.destinations.duckdb
destination_name: None
_version_hash: Ayl1W2Pj2aOzKb3gc2O7UWjqxVRpxoAIQ+4xxQjZ1IE=

sources:
Add -v option to see sources state. Note that it could be large.

Local state:
first_run: False
initial_cwd: /content
last_run_context['uri']: file:///content
_last_extracted_at: 2025-12-31 13:05:22.052194+00:00
_last_extracted_hash: Ayl1W2Pj2aOzKb3gc2O7UWjqxVRpxoAIQ+4xxQjZ1IE=

Resources in schema: github_source
github_pulls with 6 table(s) and 2 resource state slot(s)

Working dir content:
Has 3 completed load packages with following load ids:
1767185371.47367
1767186291.5332482
1767186321.0472355

Pipeline has last run

In [23]:
!pip install "dlt[workspace]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.7/266.7 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: ibis-framework
    Found existing installation: ibis-framework 9.5.0
    Uninstalling ibis-framework-9.5.0:
      Successfully uninstalled ibis-framework-9.5.0


In [22]:
# @title AI prompt cell

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown,clear_output
from google.colab import ai

dropdown = widgets.Dropdown(
    options=[],
    layout={'width': 'auto'}
)

def update_model_list(new_options):
    dropdown.options = new_options
update_model_list(ai.list_models())

text_input = widgets.Textarea(
    placeholder='Ask me anything....',
    layout={'width': 'auto', 'height': '100px'},
)

button = widgets.Button(
    description='Submit Text',
    disabled=False,
    tooltip='Click to submit the text',
    icon='check'
)

output_area = widgets.Output(
     layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
)

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=False)
        accumulated_content = ""
        for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
            if new_chunk is None:
                continue
            accumulated_content += new_chunk
            clear_output(wait=True)
            display(Markdown(accumulated_content))

button.on_click(on_button_clicked)
vbox = widgets.GridBox([dropdown, text_input, button, output_area])

display(HTML("""
<style>
.widget-dropdown select {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
.widget-textarea textarea {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
</style>
"""))
display(vbox)


GridBox(children=(Dropdown(layout=Layout(width='auto'), options=('google/gemini-2.5-flash', 'google/gemini-2.5…

---
###  **Reset State**


**To fully reset the state:**

Drop the destination dataset to fully reset the pipeline.
Set the `dev_mode` flag when creating pipeline.
Use the `dlt pipeline drop --drop-all` command to drop state and tables for a given schema name.

**To partially reset the state:**

Use the `dlt pipeline drop <resource_name>` command to drop state and tables for a given resource.
Use the `dlt pipeline drop --state-paths` command to reset the state at given path without touching the tables and data.

**Example for partial reset:**

>  in an ipynb environment, when the duckdb connection we opened is not yet closed -> close the connection before attempting to edit the pipeline through the CLI

In [28]:
conn.close()

In [29]:
!yes | dlt pipeline github_pipeline drop github_pulls

Found pipeline github_pipeline in /var/dlt/pipelines
About to drop the following data in dataset github_data in destination duckdb:
Selected schema: github_source
Selected resource(s): ['github_pulls']
Table(s) to drop: ['github_pulls__labels', 'github_pulls__requested_reviewers', 'github_pulls__assignees', 'github_pulls__base__repo__topics', 'github_pulls__head__repo__topics', 'github_pulls']
	with data in destination: ['github_pulls__labels', 'github_pulls__requested_reviewers', 'github_pulls__assignees', 'github_pulls__base__repo__topics', 'github_pulls__head__repo__topics', 'github_pulls']
Resource(s) state to reset: ['github_pulls']
Source state path(s) to reset: []


In [30]:
!dlt pipeline -v github_pipeline info

Attaching to pipeline github_pipeline
Found pipeline github_pipeline in /var/dlt/pipelines
Synchronized state:
_state_version: 3
_state_engine_version: 4
dataset_name: github_data
schema_names: ['github_source']
pipeline_name: github_pipeline
default_schema_name: github_source
destination_type: dlt.destinations.duckdb
destination_name: None
_version_hash: Ayl1W2Pj2aOzKb3gc2O7UWjqxVRpxoAIQ+4xxQjZ1IE=

sources:
{
  "github_source": {
    "resources": {
      "github_pulls": {
        "incremental": {
          "updated_at": {
            "initial_value": "2024-12-01",
            "last_value": "2025-12-30T13:39:21Z",
            "unique_hashes": [
              "pkjQiUBfoMIwJumCQqtv"
            ]
          },
          "second_new_key": "fourth_value"
        },
        "new_key": [
          "first_value",
          "second_value",
          "third_value",
          "third_value"
        ]
      }
    }
  }
}

Local state:
first_run: False
initial_cwd: /content
last_run_context['uri']:

In [31]:
!dlt pipeline github_pipeline trace

Found pipeline github_pipeline in /var/dlt/pipelines
Run started at 2025-12-31 13:06:31.991611+00:00 and COMPLETED in 1.23 seconds with 4 steps.
Step extract COMPLETED in 1.07 seconds.

Load package 1767186392.1209395 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.04 seconds.
No data found to normalize

Step load COMPLETED in 0.03 seconds.
Pipeline github_pipeline load step completed in ---
0 load package(s) were loaded to destination duckdb and into dataset None
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data

Step run COMPLETED in 1.23 seconds.
Pipeline github_pipeline load step completed in ---
0 load package(s) were loaded to destination duckdb and into dataset None
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data


🎊🎊🎊 That is actually it! We hope you enjoyed this course and learned more about dlt! 🎊🎊🎊

Please share your feedback with us: [Feedback Google Form](https://forms.gle/1NYrGcRj5gLQ4WDt8) 🌼